In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import os
from processingmm import batch_processing
from histopolalign import combine_images, prepare_images, semi_automatic_tool, align_folders, match_skeletons, align_imgs, finalize_mask, get_numerical_results

# 1. Process the measurements that will be aligned

In [2]:
# get the folder in which the polarimetric measurements are stored
neoplastic_polarimetry_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'TumorMeasurements')
neoplastic_folder = '55'
polarimetry_path = os.path.join(neoplastic_polarimetry_path, neoplastic_folder)
directories = [os.path.join(neoplastic_polarimetry_path, neoplastic_folder)]

In [3]:
# process the measurements, if needed
calib_directory = os.path.join(os.getcwd().split('notebooks')[0], 'calib')
# batch_processing.batch_process(directories, calib_directory, run_all = False)

# 2. Get the combined masks for the histology folders

In [4]:
# create the folde that will be used to align pathology and polarimetry
combine_images.create_to_align_folder()

# load the pathology images...
histology_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'HistologyResults')
imgs_all = combine_images.load_the_images(histology_path, Verbose = True)

# ...and process them
combine_images.get_combined_img(imgs_all, force_recompute = True, Verbose = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:02<00:00, 17.75it/s]


Load the images: 2.328 seconds.
Get combined images: 0.895 seconds.


# 3. Obtain the parameters (manually using the GUI) to first align the histology and the polarimetry

### 3.1. Prepare the images

In [5]:
alignment_measurements = prepare_images.create_the_alignments(histology_path, polarimetry_path, Verbose = True)
alignment_measurements

Get polarimetry filenames and rename images: 0.008 seconds.
Create the alignment objects: 0.272 seconds.


[Folder path: C:\Users\romai\Documents\PathologyPaper\data\TumorMeasurements\55\2022-10-11_T_HORAO-55-BF_FR_S1_1, histology path : C:\Users\romai\Documents\PathologyPaper\data\HistologyResults\55\HORAO-55-S1-1,  status: initialized]

### 3.2. Load the polarimetry, labels and histology (H&E) images for each polarimetry folder

In [6]:
alignment_measurements = prepare_images.load_and_preprocess_imgs(alignment_measurements, force_recompute = False, Verbose = True)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Get the images and center of mass: 1.335 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.59s/it]

Load the images: 0.255 seconds.


### 3.3. Semi-automatic rotation/alignement tool

In [7]:
for measurement in alignment_measurements:
    selector = semi_automatic_tool.ask_for_parameters(measurement, force_recompute = False)

# 4. Actually perform the alignment

In [8]:
align_folders.create_align_folders(alignment_measurements, Verbose = True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.28it/s]

Process the images to apply the selected changes: 0.209 seconds.
Save the processed images: 0.024 seconds.


### 4.1. Automatic part

In [9]:
border_parameter = 5
nsamples = 400
max_distance = 150

for measurement in tqdm(alignment_measurements):
    match_skeletons.match_skeletons(measurement, border_parameter, nsamples = nsamples, max_distance = max_distance,
                                                                   Verbose = True)

  0%|                                                                                            | 0/1 [00:03<?, ?it/s]

KeyboardInterrupt



### 4.2. Semi-automatic part

In [ ]:
align_folders.semi_automatic_processing(alignment_measurements)

In [ ]:
alignment_measurements = align_imgs.align_img_master(alignment_measurements)

### 4.3. Get the final masks and overlay them on the polarimetric parameter maps

In [ ]:
finalize_mask.generate_final_masks(alignment_measurements, Verbose = False)

### 4.4. Finalize and save the numerical results

In [ ]:
values = get_numerical_results.create_numerical_values(alignment_measurements)